In [1]:
import pandas as pd
import numpy as np
import datetime
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df = pd.read_csv("data/bikes.csv", index_col=0)

<p> Eliminamos la columna index al tratarse de un índice </p>

In [3]:
df.drop(['instant'], axis= 1, inplace= True)

<p> Cambiamos la columna dteday a datetime </p>

In [4]:
df["dteday"]=df["dteday"].astype('datetime64[ns]')

In [6]:
df.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2018-01-01,spring,0,1,0,6,0,2,14.11,18.18,80.58,10.75,331,654,985
1,2018-02-01,spring,0,1,0,0,0,2,14.90,17.69,69.61,16.65,131,670,801
2,2018-03-01,spring,0,1,0,1,1,1,8.05,9.47,43.73,16.64,120,1229,1349
3,2018-04-01,spring,0,1,0,2,1,1,8.20,10.61,59.04,10.74,108,1454,1562
4,2018-05-01,spring,0,1,0,3,1,1,9.31,11.46,43.70,12.52,82,1518,1600


In [ ]:
df(df["dtday"]> 2018-01-01 | df["dteday"]>)

<p> Guardar los cambios en este archivo al final de la sesión </p>

In [5]:
df.to_pickle("data/bikes_limpieza.pkl")